# Import packages

In [1]:
from __future__ import division, print_function # Make these codes be compatible with Python 2
import time
import numpy as np
import pandas as pd

# Read tables into dataframes

In [2]:
dfa = pd.read_csv('files/table_A.csv')
dfb = pd.read_csv('files/table_B.csv')

In [3]:
dfa.head()

,ra,dec,z,specObjID,objid,u_mag,g_mag,r_mag,i_mag,z_mag
0,187.27970,-1.193372,0.032031,3.250000e+17,1.240000e+18,19.24800,18.01970,17.60396,17.41241,17.15981
1,210.76184,-1.094643,0.032493,3.400000e+17,1.240000e+18,18.59815,17.87038,17.66903,17.52236,17.59061
2,185.74969,-0.293052,0.030245,3.250000e+17,1.240000e+18,19.02032,18.05203,17.60972,17.43984,17.47915
3,193.52240,0.175311,0.022748,3.290000e+17,1.240000e+18,18.30348,16.88793,16.19267,15.73816,15.40265
4,222.84758,0.007093,0.023165,3.480000e+17,1.240000e+18,17.55991,15.81113,15.03432,14.67037,14.45513


In [4]:
dfb.head()

,wise_ra,wise_dec,sdss_objid,w1,w1err,w1snr,w2,w2err,w2snr,w3,w3err,w3snr,w4,w4err,w4snr
0,0.697183,29.359842,1237663307454874339,14.330,0.030,36.7,14.054,0.046,23.6,12.702,9999.000,-1.5,9.230,9999.0,-1.1
1,359.329741,32.799931,1237663306916430563,17.363,0.190,5.7,17.097,9999.000,-0.2,12.429,9999.000,1.0,9.065,9999.0,0.5
2,1.379313,24.340139,1237663306920231248,17.354,0.177,6.1,16.286,0.314,3.5,12.748,9999.000,-0.6,8.721,9999.0,1.0
3,2.291314,24.765421,1237663307993842580,16.353,0.074,14.7,16.233,0.281,3.9,12.498,9999.000,-0.6,8.351,9999.0,1.6
4,2.475097,26.254814,1237663308530122881,13.670,0.027,40.8,13.599,0.038,28.6,11.877,0.246,4.4,9.057,9999.0,0.4


In [5]:
print('Table A has', len(dfa), 'objects.')
print('Table B has', len(dfb), 'objects.')

Table A has 59574 objects.
Table B has 129350 objects.


In [6]:
ra_a = dfa['ra'].values
dec_a = dfa['dec'].values

ra_b = dfb['wise_ra'].values
dec_b = dfb['wise_dec'].values

# 1. Naive cross-matcher

### Use for loop to do cross-match
#### $\mathrm{Distance} = \sqrt{(RA_{ai}-RA_{bi})^{2}+(Dec_{ai}-Dec_{bi})^{2}}$

In [7]:
# Start time
t1_0 = time.time()

#------------------------------------
idx = []
for i in range(len(dfa[:10])):
    dist = []
    for j in range(len(dfb)):
        # Calcuate the distance between target from dfa and all targets from dfb
        dist.append(np.sqrt((ra_a[i] - ra_b[j])**2 + (dec_a[i] - dec_b[j])**2))
    # Check whether the minimum is less than the given radius (1 arcsec)
    if min(dist) < (1. / 3600.):
        idx.append(dist.index(min(dist)))
    # Otherwise, set the matching index to -9999 for not matched
    else:
        idx.append(-9999)
#------------------------------------

# End time
t1_1 = time.time()
# Time interval for matching two targets
dt1 = t1_1 - t1_0
# Time interval scaled to all targets
dt1t = dt1 / (len(dfa[:10]) / len(dfa))

In [8]:
print('Time for matching 10 targets:', round(dt1, 2), 'seconds')
print('Time for matching all targets:', round(dt1t / 3600, 2), 'hours')

Time for matching 10 targets: 7.05 seconds
Time for matching all targets: 11.66 hours


# 2. Numpy array cross-matcher

In [9]:
# Start time
t2_0 = time.time()

# Array filled up with zero as the same dimension as ra_a
idx = np.zeros_like(ra_a)
dist = np.zeros_like(ra_a)

for i in range(len(dfa[:2500])):
    # Use ``numpy.where`` to find out targets within the given radius (1 arcsec)
    ind = np.where(np.sqrt((ra_b - ra_a[i])**2 + (dec_b - dec_a[i])**2) < (1. / 3600.))[0]
    if ind.size == 0:
        idx[i] = -9999
        dist[i] = -9999
    else:
        idx[i] = ind[0]
        dist[i] = np.sqrt((ra_b[ind[0]] - ra_a[i])**2 + (dec_b[ind[0]] - dec_a[i])**2)

# End time
t2_1 = time.time()
# Time interval for matching 5000 targets
dt2 = t2_1 - t2_0
# Time interval scaled to all targets
dt2t = dt2 / (len(dfa[:2500]) / len(dfa))

In [10]:
print('Time for matching 2500 targets:', round(dt2, 2), 'seconds')
print('Time for matching all targets:', round(dt2t, 2), 'seconds')

Time for matching 2500 targets: 6.9 seconds
Time for matching all targets: 164.39 seconds


# 3. Astropy SkyCoord cross-matcher

In [11]:
from astropy.coordinates import SkyCoord
from astropy import units as u

In [12]:
# Start time
t3_0 = time.time()

# Convert ra, dec to SkyCoord
ca = SkyCoord(ra_a*u.deg, dec_a*u.deg)
cb = SkyCoord(ra_b*u.deg, dec_b*u.deg)

# Matching
idx, d2d, d3d = ca.match_to_catalog_sky(cb)
matches = cb[idx]
dra, ddec = ca.spherical_offsets_to(matches)

# Set a matching radius
idx_new = idx[d2d < 1 * u.arcsec]

# End time
t3_1 = time.time()
dt3 = t3_1 - t3_0

In [13]:
print('Time for matching all targets:', round(dt3, 2), 'seconds')

Time for matching all targets: 0.49 seconds


# 4. Astroml cross-matcher

In [14]:
from astroML.crossmatch import crossmatch_angular

In [15]:
# Start time
t4_0 = time.time()

imX = np.vstack((ra_a, dec_a)).T

stX = np.vstack((ra_b, dec_b)).T

# crossmatch catalogs
max_radius = 1. / 3600
dist, idx = crossmatch_angular(imX, stX, max_radius)
match = ~np.isinf(dist)

# Set those not matched to -9999
idx[~match] = -9999

# End time
t4_1 = time.time()
dt4 = t4_1 - t4_0

In [16]:
print('Time for matching all targets:', round(dt4, 2), 'seconds')

Time for matching all targets: 0.23 seconds


# Time

In [17]:
print('1. Naive cross-matcher:\t\t\t{} hours \t {} times faster than "2. Numpy array cross-matcher"'.format(round(dt1t / 3600, 2), round(dt1t/dt2t, 2)))
print('2. Numpy array cross-matcher:\t\t{} mins \t {} times faster than "3. Astropy SkyCoord cross-matcher"'.format(round(dt2t / 60, 2), round(dt2t/dt3, 2)))
print('3. Astropy SkyCoord cross-matcher:\t{} secs \t {} times faster than "4. Astroml cross-matcher"'.format(round(dt3, 2), round(dt3/dt4, 2)))
print('4. Astroml cross-matcher:\t\t{} secs'.format(round(dt4, 2)))

1. Naive cross-matcher:			11.66 hours 	 255.41 times faster than "2. Numpy array cross-matcher"
2. Numpy array cross-matcher:		2.74 mins 	 337.5 times faster than "3. Astropy SkyCoord cross-matcher"
3. Astropy SkyCoord cross-matcher:	0.49 secs 	 2.16 times faster than "4. Astroml cross-matcher"
4. Astroml cross-matcher:		0.23 secs
